In [1]:
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import csv
import numpy as np

sia =SentimentIntensityAnalyzer()
def get_sentiment_of(sentence_in):
    try:
        scrs=sia.polarity_scores(sentence_in)
        the_important_one=scrs['compound']
    except:
        the_important_one=0
    return the_important_one

df = pd.read_csv('yelp_reviews_chicago_labeled.csv')
df2 = df.drop(['reviewID', 'restaurantID', 'reviewerID', 'date', 'name', 'location', 'yelpJoinDate', 'mnr', 'rl', 'rd', 'Maximum Content Similarity'], axis = 1)
df2 = df2.drop(['friendCount', 'reviewCount', 'firstCount', 'usefulCount', 'complimentCount', 'tipCount', 'fanCount', 'restaurantRating'], axis = 1)
f = df2['reviewContent']
sent = list()
for rows in f:
    #print(rows)
    sent.append(get_sentiment_of(rows))
print(len(sent))

df2['Sentiment'] = sent
df_final = df2.drop(['reviewContent'], axis = 1)
fields_with_nan_length = df_final.isna().sum().where(lambda x : x > 0).dropna()
df_final = df_final.iloc[:-1, :]
authentic_reviews_df = df_final[df_final['flagged'] == 'N']
fake_reviews_df = df_final[df_final['flagged'] == 'Y']
authentic_reviews_df = authentic_reviews_df.sample(n = 500)
df_final = pd.concat([authentic_reviews_df , fake_reviews_df])
df_final['flagged'] = [0 if x == 'N' else 1 for x in df_final['flagged']]
flagged = df_final['flagged']
df_final = df_final.drop(['flagged'], axis=1)
df_final = pd.concat([df_final,flagged], axis=1)
X = df_final.iloc[:, :-1]
y = flagged
X.head()



3626


,rating,reviewUsefulCount,coolCount,funnyCount,Sentiment
1646,5,0,1.0,0.0,0.9702
3375,5,0,2.0,0.0,0.8860
723,4,0,6.0,21.0,0.9862
3040,5,1,3.0,3.0,0.9891
1344,4,0,1.0,0.0,0.9871


In [2]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)

In [4]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix

print("phase1")
svc = SVC(kernel = 'linear', gamma = 1.0)
print("phase2")
svc.fit(X_train, y_train)
print("phase3")
y_pred = svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test, y_pred))

phase1
phase2
phase3
Accuracy Score:
0.8205128205128205


In [5]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_test, y_pred))
dt_cm = confusion_matrix(y_test,y_pred )

              precision    recall  f1-score   support

           0       0.99      0.66      0.79       100
           1       0.73      0.99      0.84        95

   micro avg       0.82      0.82      0.82       195
   macro avg       0.86      0.82      0.82       195
weighted avg       0.86      0.82      0.82       195

